Metro Census
============
This notebook gathers US census data from the American Community Survey (ACS) 5-year data set
and prepares it in straightforward data files for analysis, focusing on cities and metro
areas in the US.


In [1]:

from census import Census
import us
from us import states
import os
from types import SimpleNamespace
import pandas as pd
import geopandas as gpd

api_key = os.environ["CENSUS_API_KEY"]

abs(-8)

8

In [2]:

# use these to give human readable names to census variables
census_pretty = {
    "B01003_001E": "Total Population",
    "B19013_001E": "Median Household Income",
    "B25001_001E": "Total Housing Units",
    "B25002_002E": "Occupied Housing Units",
    "B03002_001E": "Total Population - Race",
    "B03002_003E": "White Alone",
    "B03002_004E": "Black or African American Alone",
    "B03002_012E": "Hispanic or Latino",
    "B17001_002E": "Poverty Status - Below Poverty Level",
    "B22002_002E": "Households - With Children",
    "P008003": "White Population",
    "NAME": "Name",
    "B08012_001E": "# of Commuters",
    "B08012_002E": "Less than 5 Minutes",
    "B08012_003E": "5-9 Minutes",
    "B08012_004E": "10-14 Minutes",
    "B08012_005E": "15-19 Minutes",
    "B08012_006E": "20-24 Minutes",
    "B08012_007E": "25-29 Minutes",
    "B08012_008E": "30-34 Minutes",
    "B08012_009E": "35-39 Minutes",
    "B08012_010E": "40-44 Minutes",
    "B08012_011E": "45-59 Minutes",
    "B08012_012E": "60-89 Minutes",
    "B08012_013E": ">90 Minutes",
    "GEO_ID": "GEOID",
    "metropolitan statistical area/micropolitan statistical area": "MSA"
}

# dot notation access to census variables
cv = {
    "name": "NAME",
    "total_pop": "B01003_001E",
    "median_inc": "B19013_001E",
    "total_housing": "B25001_001E",
    "occupied_housing": "B25002_002E",
    "total_pop_race": "B03002_001E",
    "asian": "B03002_006E",
    "black": "B03002_004E",
    "indian": "B03002_005E",
    "latino": "B03002_012E",
    "mixed": "B03002_009E",
    "other": "B03002_008E",
    "pacific": "B03002_007E",
    "white": "B03002_003E",
    "white_pop": "P008003",
    "poverty": "B17001_002E",
    "geoid": "GEO_ID"
}
cv = SimpleNamespace(**cv)
vars = [v for k,v in cv.__dict__.items()]
vars

['NAME',
 'B01003_001E',
 'B19013_001E',
 'B25001_001E',
 'B25002_002E',
 'B03002_001E',
 'B03002_006E',
 'B03002_004E',
 'B03002_005E',
 'B03002_012E',
 'B03002_009E',
 'B03002_008E',
 'B03002_007E',
 'B03002_003E',
 'P008003',
 'B17001_002E',
 'GEO_ID']

In [3]:
# get the census data from MSAs
c = Census(api_key)
data = c.acs5.get("B01003", {'for': 'metropolitan statistical area:*'})
data

CensusException: error: unknown/unsupported geography heirarchy

In [20]:


# Fetch data for all MSAs (metropolitan statistical areas)
years = list(range(2022, 2016, -1))
print(years)
def data_year(year):
    c = Census(api_key, year=year)
    df = c.acs5.get([cv.name, cv.total_pop, cv.geoid], {'for': 'place:*'})
    return df


data = [pd.DataFrame(data_year(year)) for year in years]
# c = Census(api_key, year=2023)
# data = c.acs5.get([cv.name, cv.total_pop, cv.geoid], {'for': 'place:*'})
data

[2022, 2021, 2020, 2019, 2018, 2017]


[                                   NAME  B01003_001E            GEO_ID state  \
 0                   Abanda CDP, Alabama        335.0  1600000US0100100    01   
 1               Abbeville city, Alabama       2309.0  1600000US0100124    01   
 2              Adamsville city, Alabama       4325.0  1600000US0100460    01   
 3                 Addison town, Alabama        665.0  1600000US0100484    01   
 4                   Akron town, Alabama        310.0  1600000US0100676    01   
 ...                                 ...          ...               ...   ...   
 32181  Voladoras comunidad, Puerto Rico        809.0  1600000US7287638    72   
 32182  Yabucoa zona urbana, Puerto Rico       6082.0  1600000US7287863    72   
 32183    Yauco zona urbana, Puerto Rico      15165.0  1600000US7288035    72   
 32184     Yaurel comunidad, Puerto Rico        646.0  1600000US7288121    72   
 32185    Yeguada comunidad, Puerto Rico       1756.0  1600000US7288293    72   
 
        place  
 0      00

In [24]:


cities = pd.DataFrame(data[0])
cities.columns = ["name", "pop_2022", "geoid", "state_fips", "place"]
cities["state"] = cities.state_fips.map(us.states.mapping('fips', 'abbr'))
cities.sort_values("pop_2022", ascending=False, inplace=True)
cities = cities[cities.pop_2022 > 100_000].copy()
cities.pop_2022 = cities.pop_2022.astype(int)
for i, df in enumerate(data[1:], start=1):
    year = years[i]
    pop = f"pop_{year}"
    df.columns = ["name", pop, "geoid", "state_fips", "place"]
    df[pop] = df[pop].astype(int)
    cities = cities.merge(df[["geoid", pop]], on="geoid", how="inner")

# y12 = data[-1]
# y12[y12.place == "00100"]
cities

,name,pop_2022,geoid,state_fips,place,state,pop_2021,pop_2020,pop_2019,pop_2018,pop_2017
0,"New York city, New York",8622467,1600000US3651000,36,51000,NY,8736047,8379552,8419316,8443713,8560072
1,"Los Angeles city, California",3881041,1600000US0644000,06,44000,CA,3902440,3973278,3966936,3959657,3949776
2,"Chicago city, Illinois",2721914,1600000US1714000,17,14000,IL,2742119,2699347,2709534,2718555,2722586
3,"Houston city, Texas",2296253,1600000US4835000,48,35000,TX,2293288,2313238,2310432,2295982,2267336
4,"Phoenix city, Arizona",1609456,1600000US0455000,04,55000,AZ,1591119,1658422,1633017,1610071,1574421
...,...,...,...,...,...,...,...,...,...,...,...
334,"San Tan Valley CDP, Arizona",101207,1600000US0464210,04,64210,AZ,96127,104936,96692,93230,90665
335,"Quincy city, Massachusetts",100981,1600000US2555745,25,55745,MA,100544,94389,94207,94121,93824
336,"Edinburg city, Texas",100964,1600000US4822660,48,22660,TX,98759,97734,95847,94019,86123
337,"Lynn city, Massachusetts",100653,1600000US2537490,25,37490,MA,100233,94201,93743,93617,93069


In [23]:
df = df.sort_values(by="population", ascending=False)
cities = df[df.population > 100_000].copy()
cities.population = cities.population.astype(int)

cities["city"] = cities["name"].str.split(",", expand=True)[0].replace(" city", "", regex=True)
cities = cities[["city", "state", "population"]]
cities.to_csv("data/us-cities.csv", index=False)


KeyError: 'population'

In [62]:
places = [
"tl_2023_01_place.geojson", "tl_2023_02_place.geojson", "tl_2023_04_place.geojson", "tl_2023_05_place.geojson",
"tl_2023_06_place.geojson", "tl_2023_08_place.geojson", "tl_2023_09_place.geojson", "tl_2023_10_place.geojson", 
"tl_2023_11_place.geojson", "tl_2023_12_place.geojson", "tl_2023_13_place.geojson", "tl_2023_15_place.geojson",
"tl_2023_16_place.geojson", "tl_2023_17_place.geojson", "tl_2023_18_place.geojson", "tl_2023_19_place.geojson",
"tl_2023_20_place.geojson", "tl_2023_21_place.geojson", "tl_2023_22_place.geojson", "tl_2023_23_place.geojson",
"tl_2023_24_place.geojson", "tl_2023_25_place.geojson", "tl_2023_26_place.geojson", "tl_2023_27_place.geojson",
"tl_2023_28_place.geojson", "tl_2023_29_place.geojson", "tl_2023_30_place.geojson", "tl_2023_31_place.geojson",
"tl_2023_32_place.geojson", "tl_2023_33_place.geojson", "tl_2023_34_place.geojson", "tl_2023_35_place.geojson",
"tl_2023_36_place.geojson", "tl_2023_37_place.geojson", "tl_2023_38_place.geojson", "tl_2023_39_place.geojson",
"tl_2023_40_place.geojson", "tl_2023_41_place.geojson", "tl_2023_42_place.geojson", "tl_2023_44_place.geojson",
"tl_2023_45_place.geojson", "tl_2023_46_place.geojson", "tl_2023_47_place.geojson", "tl_2023_48_place.geojson",
"tl_2023_49_place.geojson", "tl_2023_50_place.geojson", "tl_2023_51_place.geojson", "tl_2023_53_place.geojson",
"tl_2023_54_place.geojson", "tl_2023_55_place.geojson", "tl_2023_56_place.geojson", "tl_2023_60_place.geojson",
"tl_2023_66_place.geojson", "tl_2023_69_place.geojson", "tl_2023_72_place.geojson", "tl_2023_78_place.geojson"
]

results = [gpd.read_file(f"../data/census_place/{f}") for f in places]
# tiger = tiger.rename(columns={"GEOID": "geoid"})
tiger = pd.concat(results)


,STATEFP,PLACEFP,PLACENS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,STATE,geometry
0,01,05932,02405250,0105932,1600000US0105932,Berry,Berry town,43,C1,N,G4110,A,27884733,15029,+33.6667018,-087.6093110,AL,"POLYGON ((-87.6391 33.66662, -87.63794 33.6666..."
1,01,25840,02403599,0125840,1600000US0125840,Fayette,Fayette city,25,C1,N,G4110,A,22143483,212108,+33.6942153,-087.8311690,AL,"POLYGON ((-87.85507 33.70778, -87.8551 33.7105..."
2,01,32536,02406632,0132536,1600000US0132536,Gu-Win,Gu-Win town,43,C1,N,G4110,A,5031289,0,+33.9443302,-087.8703760,AL,"POLYGON ((-87.88578 33.95916, -87.88577 33.959..."
3,01,02908,02403122,0102908,1600000US0102908,Ashville,Ashville city,25,C1,N,G4110,A,49762932,487106,+33.8351967,-086.2700148,AL,"POLYGON ((-86.30442 33.87221, -86.30355 33.872..."
4,01,46696,02406094,0146696,1600000US0146696,Margaret,Margaret town,43,C1,N,G4110,A,25438661,39224,+33.6728608,-086.4639420,AL,"MULTIPOLYGON (((-86.46153 33.69044, -86.461 33..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,78,27000,02651573,7827000,1600000US7827000,Coral Bay,Coral Bay CDP,57,U1,N,G4210,S,4742478,975010,+18.3451896,-064.7172899,VI,"POLYGON ((-64.73261 18.35237, -64.73247 18.352..."
6,78,28000,02414009,7828000,1600000US7828000,Cruz Bay,Cruz Bay CDP,57,U1,N,G4210,S,7828060,2463942,+18.3237480,-064.7797895,VI,"POLYGON ((-64.79981 18.3266, -64.79979 18.3266..."
7,78,65530,02651575,7865530,1600000US7865530,Red Hook,Red Hook CDP,57,U2,N,G4210,S,733480,716140,+18.3246120,-064.8412258,VI,"POLYGON ((-64.85759 18.32304, -64.8556 18.3250..."
8,78,78300,02413996,7878300,1600000US7878300,Tutu,Tutu CDP,57,U1,N,G4210,S,4235909,28670,+18.3390198,-064.8885716,VI,"POLYGON ((-64.90364 18.34551, -64.90362 18.345..."


In [65]:
# tiger = gpd.GeoDataFrame(tiger)
tiger.to_file("../data/tiger_places_2023.geojson", driver="GeoJSON")

In [25]:
# create a new (lat,long) Point for New York City
from shapely.geometry import Point
tiger = gpd.read_file("../data/tiger_places_2023.geojson")
tiger

,STATEFP,PLACEFP,PLACENS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,STATE,geometry
0,01,05932,02405250,0105932,1600000US0105932,Berry,Berry town,43,C1,N,G4110,A,27884733,15029,+33.6667018,-087.6093110,AL,"POLYGON ((-87.6391 33.66662, -87.63794 33.6666..."
1,01,25840,02403599,0125840,1600000US0125840,Fayette,Fayette city,25,C1,N,G4110,A,22143483,212108,+33.6942153,-087.8311690,AL,"POLYGON ((-87.85507 33.70778, -87.8551 33.7105..."
2,01,32536,02406632,0132536,1600000US0132536,Gu-Win,Gu-Win town,43,C1,N,G4110,A,5031289,0,+33.9443302,-087.8703760,AL,"POLYGON ((-87.88578 33.95916, -87.88577 33.959..."
3,01,02908,02403122,0102908,1600000US0102908,Ashville,Ashville city,25,C1,N,G4110,A,49762932,487106,+33.8351967,-086.2700148,AL,"POLYGON ((-86.30442 33.87221, -86.30355 33.872..."
4,01,46696,02406094,0146696,1600000US0146696,Margaret,Margaret town,43,C1,N,G4110,A,25438661,39224,+33.6728608,-086.4639420,AL,"MULTIPOLYGON (((-86.46153 33.69044, -86.461 33..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32603,78,27000,02651573,7827000,1600000US7827000,Coral Bay,Coral Bay CDP,57,U1,N,G4210,S,4742478,975010,+18.3451896,-064.7172899,VI,"POLYGON ((-64.73261 18.35237, -64.73247 18.352..."
32604,78,28000,02414009,7828000,1600000US7828000,Cruz Bay,Cruz Bay CDP,57,U1,N,G4210,S,7828060,2463942,+18.3237480,-064.7797895,VI,"POLYGON ((-64.79981 18.3266, -64.79979 18.3266..."
32605,78,65530,02651575,7865530,1600000US7865530,Red Hook,Red Hook CDP,57,U2,N,G4210,S,733480,716140,+18.3246120,-064.8412258,VI,"POLYGON ((-64.85759 18.32304, -64.8556 18.3250..."
32606,78,78300,02413996,7878300,1600000US7878300,Tutu,Tutu CDP,57,U1,N,G4210,S,4235909,28670,+18.3390198,-064.8885716,VI,"POLYGON ((-64.90364 18.34551, -64.90362 18.345..."


In [43]:

a = tiger.GEOIDFQ.unique()
b = cities[cities.state.isin(tiger.STATE)].geoid.unique()
a = set(a)
b = set(b)
b.issubset(a)
geocities = cities.merge(tiger, left_on="geoid", right_on="GEOIDFQ", how="inner")
geocities = gpd.GeoDataFrame(geocities)
geocities["area"] = geocities.ALAND * 3.861e-7
geocities["geometry"] = geocities.apply(lambda row: Point(row.INTPTLON, row.INTPTLAT), axis=1)

geocities = geocities[['NAME', 'state', 'pop_2022',  'pop_2017', 'area', 'geometry']]
geocities.rename(columns={"NAME": "city", }, inplace=True)
geocities.explore()

In [46]:
geocities.sort_values("pop_2022", ascending=False, inplace=True)
geocities.to_file("data/city-points.geojson", driver="GeoJSON")


In [101]:
geocities = geocities[['NAME', 'state','pop_2022', 'pop_2021', 'pop_2018', 'area']]
geocities.rename(columns={"NAME": "name"}, inplace=True)

In [102]:
geocities.to_csv("data/large-us-cities.csv", index=False)

In [88]:
geocities.to_crs(epsg=3857, inplace=True)

# calculate the area of geocities
geocities["area"] = geocities.geometry.area
geocities["area"] = geocities["area"] * 3.861e-7
geocities[geocities.geoid == "1600000US3651000"].explore()

In [20]:
data[['OBJECTID', 'CTLabel', 'CT2020', 'BoroCT2020',
       'CDEligibil', 'NTAName', 'NTA2020', 'CDTA2020', 'CDTANAME', 
       'Households - With Children', 'Median Household Income', 'tract',
       'county', 'name']]
income = pd.DataFrame(data.groupby("NTAName")["Median Household Income"].mean().sort_values(ascending=False)).reset_index()


,NTAName,Median Household Income
0,Upper East Side-Carnegie Hill,189470.357143
1,Financial District-Battery Park City,183866.142857
2,Tribeca-Civic Center,177522.600000
3,Brooklyn Heights,154802.833333
4,Greenwich Village,154185.428571
...,...,...
196,Brownsville,29075.428571
197,West Farms,28815.200000
198,Belmont,27838.714286
199,Tremont,25699.666667


In [ ]:

folium.Choropleth(
    geo_data=data,
    data=data,  # Your data DataFrame
    name='Median Household Income',
    columns=['GEOID', 'Median Household Income'],  # Columns from your data
    key_on='feature.properties.GEOID',  # Key that links your data to the GeoJSON
    fill_color='YlGn',  # Color scheme, e.g., 'YlGn' for yellow-green
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Legend Title',
    popup="tract info"
).add_to(base_map)
base_map
